In [1]:
import duckdb
import pandas as pd
import sqlalchemy

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///my-db.duckdb

In [30]:
%%sql
CREATE TYPE useragent_type AS ENUM ('browser', 'programmatic', 'bot', 'unknown');
CREATE TABLE useragent
 (
     string VARCHAR,
     family VARCHAR,
     type useragent_type,
     major BOOLEAN,
     PRIMARY KEY (string, family, type, major)
 )
INSERT INTO useragent SELECT * FROM read_csv_auto('useragent.csv');

""


In [4]:
%%time
%%sql
SELECT
  COUNT(*)
FROM
  uniprotkb,
  useragent
WHERE
  method = 'GET' AND
  status = 200 AND
  useragent.string = uniprotkb.useragent AND
  useragent.major = true AND
  useragent.type = 'browser'

CPU times: user 1min 6s, sys: 4.02 s, total: 1min 10s
Wall time: 20.2 s


,count_star()
0,86373098


In [6]:
%%time
%%sql
df << SELECT
  uniprotkb.datetime,
  uniprotkb.method,
  uniprotkb.request,
  uniprotkb.status,
  uniprotkb.bytes,
  uniprotkb.referrer,
  uniprotkb.useragent,
  useragent.family,
  useragent.type
FROM
  uniprotkb,
  useragent
WHERE
  method = 'GET' AND
  status = 200 AND
  request ILIKE '%ftev\_%' AND
  useragent.string = uniprotkb.useragent AND
  useragent.major = true

Returning data to local variable df
CPU times: user 5min, sys: 1min 22s, total: 6min 23s
Wall time: 8min 38s


In [8]:
df

""


In [7]:
df.loc[0,'request']

KeyError: 'request'